In [ ]:
#warnings
import warnings

# Core
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Union, Any, Callable

## Harp/Bonsai
sys.path.append('../../src/')
from bonsai import load_bonsai_config
load_bonsai_config(r"C:\git\AllenNeuralDynamics\aind-vr-foraging\Bonsai")
import harp
import harp.processing
import data_io
sys.path.append('../BehaviorAnalysis/')



In [ ]:
def load_session_data(session_path: str | Path) -> Dict[str, data_io.DataStreamSource]:
    _out_dict = {}

    HarpBehavior = harp.HarpDevice("Behavior")
    _out_dict["harp_behavior"] = data_io.HarpSource(
        device=HarpBehavior,
        path=session_path / "Behavior",
        name="behavior",
        autoload=True)
    _out_dict["software_events"] = data_io.SoftwareEventSource(
        path=session_path / "SoftwareEvents",
        name="software_events",
        autoload=True)
    _out_dict["config"] = data_io.ConfigSource(
        path=session_path / "Config",
        name="config",
        autoload=True)
    _out_dict["operation_control"] = data_io.OperationControlSource(
        path=session_path / "OperationControl",
        name="config",
        autoload=True)
    return _out_dict


In [ ]:
data_root = Path(r"Z:\scratch\vr-foraging")
sessions = [
    r"672107/20231024T130736",
    r"672106/20231024T114555",
    r"672104/20231024T103954"]


In [ ]:
streams_to_merge = ["harp_behavior", "software_events", "operation_control"]

In [ ]:
across_animal_data = []
for session_idx, session in enumerate(sessions):
    current_session = load_session_data(data_root / session)
    streams = []
    for s in streams_to_merge:
        _folder = current_session[s]
        streams += [_folder.streams[x].data for x in _folder.streams]
    concat = pd.concat(streams, axis=0)
    concat.sort_index(inplace=True)
    concat.index = concat.index.values - concat.index.values[-1]
    across_animal_data.append(concat)


In [ ]:
for s in across_animal_data:
    display(s.tail(10))